In [ ]:
import whisper

In [36]:
# speech
model = whisper.load_model("tiny")
result = model.transcribe("data/test.mp3")
#print(result["text"])
with open("data/test.txt", "w+") as f:
    f.write(result["text"])
subtitle = whisper.utils.WriteSRT.write_result(self=whisper,result=result,file='test.srt')
len(subtitle)

/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


AttributeError: 'str' object has no attribute 'write'

In [28]:
write_result(subtitles)

NameError: name 'write_result' is not defined

In [16]:
# song
model = whisper.load_model("tiny")
result = model.transcribe("data/song.mp3")
#print(result["text"])
with open("data/song.txt", "w+") as f:
    f.write(result["text"])

/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


-----------------

In [ ]:
import subprocess


In [ ]:
#OpenAI Whisper command line
    
# convert mp3 to wav file
%timeit subprocess.call(['whisper', 'data/test.mp3', '--model', 'tiny.en', '-o', 'data', '-f', 'txt'])

________


In [5]:
import speech_recognition as sr

ModuleNotFoundError: No module named 'speech_recognition'

In [ ]:
AUDIO_FILE='data/test.wav'

In [ ]:
r = sr.Recognizer()

In [ ]:
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)

In [ ]:
r.recognize_whisper(audio, model='tiny.en')

In [ ]:
"""recognizer_instance.recognize_whisper(audio_data: AudioData, model: str="base", show_dict: bool=False, load_options: Dict[Any, Any]=None, language:Optional[str]=None, translate:bool=False, **transcribe_options):"""

In [ ]:
speech_recognition.Microphone()

--------

In [ ]:
# Splitting video

#Importing library and thir function
from pydub import AudioSegment
from pydub.silence import split_on_silence

#reading from audio mp3 file
sound = AudioSegment.from_mp3("/content/Audio/song_with_silence.mp3")

# spliting audio files
audio_chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-40 )

#loop is used to iterate over the output list
for i, chunk in enumerate(audio_chunks):
   output_file = "/content/Audio/output/chunk{0}.mp3".format(i)
   print("Exporting file", output_file)
   chunk.export(output_file, format="mp3")

# chunk files saved as Output

In [39]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

audio = AudioSegment.from_file("data/test.mp4", format="mp4")

chunks = split_on_silence(audio, 
                         keep_silence=30000, 
                         min_silence_len=200, 
                         silence_thresh=-30)

for i, chunk in enumerate(chunks):
    chunk.export(f"data/Chunks/output_{i}.mp3", format="mp3")


In [130]:

from pydub import AudioSegment
from pydub.silence import split_on_silence
import whisper
import os
from datetime import timedelta
ifrom threading import Thread

%time
sound = AudioSegment.from_file("data/test.mp4", format="mp4")
chunks = split_on_silence(
    sound,

    # split on silences longer than 1000ms (1 sec)
    min_silence_len=1000,

    # anything under -16 dBFS is considered silence
    silence_thresh=-30, 

    # keep 200 ms of leading/trailing silence
    keep_silence=200
)
%time
# now recombine the chunks so that the parts are at least 30 sec long
target_length = 30 * 1000
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)

# now your have chunks that are bigger than 90 seconds (except, possibly the last one)
%time

n_threads = 8
threads = []

for i, chunk in enumerate(output_chunks):
    t = Thread(target=chunk.export(f"data/Chunks/output_{i}.mp3", format="mp3"))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

%time
# Get a list of all the audio files in the "data/Chunks/" folder
audio_files = [f for f in sorted(os.listdir("data/Chunks/")) if f.endswith('.mp3')]
%time
# Initialize an empty list to store the transcriptions
transcriptions = []
segments = []

model = whisper.load_model("tiny.en",device='cpu')
%time
# Loop over all the audio files in the "data/Chunks/" folder
for audio_file in audio_files:
    audio_file_path = os.path.join("data/Chunks/", audio_file)
    result = model.transcribe(audio_file_path)
    #transcription = str(result)
    transcriptions.append(result['text'])
    #print(len(result['segments']))
    segments.append([res for res in result['segments']])

    #os.remove(audio_file_path)
%time
with open("data/test.txt", "w+") as f:
    f.write(' '.join(transcriptions))
%time
# subtitles
# TODO:Need to fix
"""
segments_final = repairTranscriptSegments([seg for segment in segments for seg in segment],40)
#segments = result['segments']

for segment in segments_final:
    startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
    endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
    text = segment['text']
    segmentId = segment['id']+1
    segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

    srtFilename = os.path.join(r"data", "test.srt")
    with open(srtFilename, 'a', encoding='utf-8') as srtFile:
        srtFile.write(segment)
"""

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs
CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16.2 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.2 µs
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 12.4 µs


/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/si

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 15 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


'\nsegments_final = repairTranscriptSegments([seg for segment in segments for seg in segment],40)\n#segments = result[\'segments\']\n\nfor segment in segments_final:\n    startTime = str(0)+str(timedelta(seconds=int(segment[\'start\'])))+\',000\'\n    endTime = str(0)+str(timedelta(seconds=int(segment[\'end\'])))+\',000\'\n    text = segment[\'text\']\n    segmentId = segment[\'id\']+1\n    segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == \' \' else text}\n\n"\n\n    srtFilename = os.path.join(r"data", "test.srt")\n    with open(srtFilename, \'a\', encoding=\'utf-8\') as srtFile:\n        srtFile.write(segment)\n'

In [121]:
# Loop over all the audio files in the "data/Chunks/" folder
for audio_file in audio_files:
    audio_file_path = os.path.join("data/Chunks/", audio_file)
    result = model.transcribe(audio_file_path)
    #transcription = str(result)
    #transcriptions.append(result['text'])
    print(result['segments'])
    #[segments.append(res) for res in len(result['segments'])]
    segments.append([res for res in len(result['segments'])])

    #os.remove(audio_file_path)


/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.4, 'text': ' And I will share my screen.', 'tokens': [843, 314, 481, 2648, 616, 3159, 13], 'temperature': 0.0, 'avg_logprob': -0.2766423658891158, 'compression_ratio': 1.673913043478261, 'no_speech_prob': 0.07262368500232697}, {'id': 1, 'seek': 0, 'start': 3.4, 'end': 6.5200000000000005, 'text': ' So basically one of the most asked questions', 'tokens': [1406, 6209, 530, 286, 262, 749, 1965, 2683], 'temperature': 0.0, 'avg_logprob': -0.2766423658891158, 'compression_ratio': 1.673913043478261, 'no_speech_prob': 0.07262368500232697}, {'id': 2, 'seek': 0, 'start': 6.5200000000000005, 'end': 8.4, 'text': ' for this next six months arc.', 'tokens': [329, 428, 1306, 2237, 1933, 10389, 13], 'temperature': 0.0, 'avg_logprob': -0.2766423658891158, 'compression_ratio': 1.673913043478261, 'no_speech_prob': 0.07262368500232697}, {'id': 3, 'seek': 0, 'start': 8.4, 'end': 9.4, 'text': ' Can you all see my screen?', 'tokens': [1680, 345, 477, 766, 616, 315

TypeError: 'int' object is not iterable

In [122]:
#' '.join(transcriptions)
seg_final = [seg for segment in segments for seg in segment]
len(seg_final)

63

In [124]:
def repairTranscriptSegments(transcript, chunkSeconds):
    """
        We are repairing the timeline here.
        If we are just transcibing, repair of the timeline is not needed because
        we would simply not use the timeline information but if we want to develop
        an SRT, VTT or some other tool that makes use of timing, this is required.
        We expect that for every new audio chunk, the segment information will be
        re-initialized.  We concatenated each chunk's segments and determining an
        offset amount for each chunk, then adjust the "start" and "end" segments,
        so the timestamps are contiguous.
        "chunkSeconds" is the length of each chunk, so we know that the segment["end"]
        cant be beyond this. After repairing the timeline, from the concatenated chunks
        we can use the built-in functions to write TEXT, SRT and VTT files.
    """
    
    chunkCount = 0
    for segIndex, segment in enumerate(transcript):
        if (segment["id"] == 0):
            chunkCount += 1
            if (chunkCount > 1):
                prevSegment = transcript[segIndex-1] # look back to previous segment
                if (prevSegment["end"] > chunkSeconds): # impossible to be greater than chunk length
                    transcript[segIndex-1]["end"] = chunkSeconds

    chunkCount = 0 # re-initialize
    adjustTime = 0.0
    for segIndex, segment in enumerate(transcript):
        segID = segment["id"]
        if (segID == 0):
            chunkCount += 1
            adjustTime  = ((chunkCount - 1) * chunkSeconds)
                
#        if (segID == 0)   : segment["text"] += "@@@@@" # TESTING: This is a marker so we know where each new segment is located
        segment["start"] += adjustTime
        segment["end"]   += adjustTime
        segment["id"]     = segIndex # repair the sequence, so the IDs are now contiguous
    return transcript

In [125]:
repairTranscriptSegments(segments_final,30)

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.4,
  'text': ' And I will share my screen.',
  'tokens': [843, 314, 481, 2648, 616, 3159, 13],
  'temperature': 0.0,
  'avg_logprob': -0.2766423658891158,
  'compression_ratio': 1.673913043478261,
  'no_speech_prob': 0.07262368500232697},
 {'id': 1,
  'seek': 0,
  'start': 3.4,
  'end': 6.5200000000000005,
  'text': ' So basically one of the most asked questions',
  'tokens': [1406, 6209, 530, 286, 262, 749, 1965, 2683],
  'temperature': 0.0,
  'avg_logprob': -0.2766423658891158,
  'compression_ratio': 1.673913043478261,
  'no_speech_prob': 0.07262368500232697},
 {'id': 2,
  'seek': 0,
  'start': 6.5200000000000005,
  'end': 8.4,
  'text': ' for this next six months arc.',
  'tokens': [329, 428, 1306, 2237, 1933, 10389, 13],
  'temperature': 0.0,
  'avg_logprob': -0.2766423658891158,
  'compression_ratio': 1.673913043478261,
  'no_speech_prob': 0.07262368500232697},
 {'id': 3,
  'seek': 0,
  'start': 8.4,
  'end': 9.4,
  'text': ' Can 